<a href="https://colab.research.google.com/github/sudh042001/ML_Project/blob/master/Fake_news_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FAKE NEWS CLASSIFIER USING LSTM

DataSet:https://www.kaggle.com/c/fake-news

In [1]:
import pandas as pd

In [13]:
df=pd.read_csv('train.csv')

In [14]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [15]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [16]:
df.shape

(20800, 5)

In [17]:
df=df.dropna()

In [18]:
X=df.drop('label',axis=1)

In [19]:
y=df['label']

In [20]:
X.shape

(18285, 4)

In [21]:
y.shape

(18285,)

In [22]:
import tensorflow as tf


In [23]:
tf.__version__

'2.9.2'

In [25]:
messages=X.copy()

In [26]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [28]:
messages.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [29]:
messages.reset_index(inplace=True)
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [31]:
import nltk
import re
from nltk.corpus import stopwords

Text Cleaning & Preprocessing

In [32]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [33]:
##data preprocessing 
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
  review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review=review.lower()
  review=review.split()

  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [ ]:
corpus

In [24]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [35]:
voc_size=5000

Onehot Representation

In [36]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]

In [ ]:
onehot_repr

Padding

In [38]:
sent_length=20
embedded_doc=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

In [40]:
embedded_doc

array([[   0,    0,    0, ..., 1129, 1935, 2548],
       [   0,    0,    0, ..., 1431, 1072, 3470],
       [   0,    0,    0, ..., 4047, 3679, 1186],
       ...,
       [   0,    0,    0, ..., 2092, 2857, 2031],
       [   0,    0,    0, ..., 1813, 1126, 1703],
       [   0,    0,    0, ...,   90, 1155, 4060]], dtype=int32)

Embedding

In [42]:
#creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [43]:
len(embedded_doc),y.shape

(18285, (18285,))

In [44]:
import numpy as np
X_final=np.array(embedded_doc)
y_final=np.array(y)

In [45]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

Training

In [59]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [60]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 39ms/step - loss: 0.3493 - accuracy: 0.8364 - val_loss: 0.2003 - val_accuracy: 0.9128
Epoch 2/10
192/192 [==============================] - 7s 36ms/step - loss: 0.1602 - accuracy: 0.9384 - val_loss: 0.1867 - val_accuracy: 0.9218
Epoch 3/10
192/192 [==============================] - 7s 37ms/step - loss: 0.1136 - accuracy: 0.9564 - val_loss: 0.2072 - val_accuracy: 0.9234
Epoch 4/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0832 - accuracy: 0.9695 - val_loss: 0.2241 - val_accuracy: 0.9168
Epoch 5/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0580 - accuracy: 0.9786 - val_loss: 0.2659 - val_accuracy: 0.9097
Epoch 6/10
192/192 [==============================] - 7s 36ms/step - loss: 0.0410 - accuracy: 0.9857 - val_loss: 0.2704 - val_accuracy: 0.9065
Epoch 7/10
192/192 [==============================] - 7s 36ms/step - loss: 0.0344 - accuracy: 0.9886 - val_loss: 0.3243 - val_accuracy: 0.910

Adding Dropout

In [58]:
from tensorflow.keras.layers import Dropout
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


Performance Metrics And Accuracy

In [61]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 6ms/step


In [62]:
y_pred=np.where(y_pred >0.5,1,0)

In [63]:
from sklearn.metrics import confusion_matrix

In [64]:
confusion_matrix(y_test,y_pred)

array([[3105,  314],
       [ 239, 2377]])

In [65]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9083678541839271

In [66]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.88      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

